In [3]:
# K-Nearest Neighbor Classification
 
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from skimage import exposure
import numpy as np
#import imutils
import cv2 as cv
import os
from os import listdir, makedirs
from os.path import isfile, join,basename
 
from PIL import Image
desired_size = 128
 
def process_image(old_im_pth):
    im = Image.open(old_im_pth)
    old_size = im.size
 
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
 
    im = im.resize(new_size, Image.ANTIALIAS)
 
    new_im = Image.new("RGB", (desired_size, desired_size), (255, 225, 255))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                      (desired_size-new_size[1])//2))
    return np.array(new_im)
 
#lookup = {'axes': 1, 'boots': 2, 'carabiners': 3, 'crampons': 4, 'gloves': 5,}
#r_lookup = {1: 'axes', 2: 'boots', 3: 'carabiners', 4: 'crampons', 5: 'gloves'}
lookup = {'axes': 1, 'boots': 2, 'carabiners': 3, 'crampons': 4, 'gloves': 5, 'hardshell_jackets': 6, 'harnesses': 7, 'helmets': 8, 'insulated_jackets': 9, 'pulleys': 10, 'rope': 11, 'tents': 12}
r_lookup = {1: 'axes', 2: 'boots', 3: 'carabiners', 4: 'crampons', 5: 'gloves', 6: 'hardshell_jackets', 7: 'harnesses', 8: 'helmets', 9: 'insulated_jackets', 10: 'pulleys' , 11: 'rope', 12: 'tents'}
img_root = '/home/team12/naresh/gear_images'
img_dirs = [os.path.join(img_root, o) for o in os.listdir(
    img_root) if os.path.isdir(os.path.join(img_root, o))]
class_index = 1
dataset = np.empty((1, desired_size * desired_size * 3))
labelset = np.empty(1)
for img_dir in img_dirs:
    img_files = [f for f in listdir(img_dir) if isfile(join(img_dir, f))]
    image_label = np.full([len(img_files)], lookup[basename(img_dir)])
    print(image_label)
    image_data = np.empty([len(img_files), desired_size * desired_size * 3])
    index = 0
    for img_file in img_files:
        image_path = img_dir + '/' + img_file
        dst = np.zeros(shape=(desired_size, desired_size))
        image = process_image(image_path)
        norm_image = cv.normalize(image, dst, 0, 255, cv.NORM_MINMAX)
        image_data[index] = norm_image.flatten()
        # print(image_data.shape)
        index = index + 1
    class_index = class_index + 1
    dataset = np.concatenate((dataset, image_data), axis=0)
    labelset = np.concatenate((labelset, image_label), axis=0)
 
print(dataset.astype('uint8').dtype)
print(labelset.astype('uint8').dtype)
 
(trainData, testData, trainLabels, testLabels) = train_test_split(
    dataset.astype('uint8'), labelset.astype('uint8'), test_size=0.20, random_state=42)
 
(trainData, valData, trainLabels, valLabels) = train_test_split(
    trainData, trainLabels, test_size=0.1, random_state=84)
 
# Checking sizes of each data split
print("training data points: {}".format(len(trainLabels)))
print("validation data points: {}".format(len(valLabels)))
print("testing data points: {}".format(len(testLabels)))
 
kVals = range(1, 30, 2)
accuracies = []
 
# loop over kVals
for k in range(1, 30, 2):
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainData, trainLabels)
 
    # evaluate the model and print the accuracies list
    score = model.score(valData, valLabels)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)
 
# largest accuracy
# np.argmax returns the indices of the maximum values along an axis
i = np.argmax(accuracies)
print("k=%d achieved highest accuracy of %.2f%% on validation data" % (kVals[i],
                                                                       accuracies[i] * 100))
 
# Now that I know the best value of k, re-train the classifier
model = KNeighborsClassifier(n_neighbors=kVals[i])
model.fit(trainData, trainLabels)
 
# Predict labels for the test set
predictions = model.predict(testData)
 
# Evaluate performance of model for each of the digits
print("EVALUATION ON TESTING DATA")
print(classification_report(testLabels, predictions))
from PIL import Image
for i in np.random.randint(0, high=len(testLabels), size=(5,)):
    image = testData[i]
    prediction = model.predict(image.reshape(1, -1))[0]
    # show the prediction
    print("Image Name {}.png".format(i))
    print("I think that image is: {}".format(r_lookup[prediction]))
    #img = Image.fromarray(testData[i].reshape(128,128,3), 'RGB')
    #img.show()

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2]
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from skimage import exposure
import numpy as np
#import imutils
import cv2 as cv
import os
from os import listdir, makedirs
from os.path import isfile, join,basename
import matplotlib as plt
from PIL import Image
 
from PIL import Image
desired_size = 128
 
def process_image(old_im_pth):
    im = Image.open(old_im_pth)
    old_size = im.size
 
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
 
    im = im.resize(new_size, Image.ANTIALIAS)
 
    new_im = Image.new("RGB", (desired_size, desired_size), (255, 225, 255))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                      (desired_size-new_size[1])//2))
    return np.array(new_im)
 
def normalize(image_path):
    dst = np.zeros(shape=(desired_size, desired_size))
    image = process_image(image_path)
    norm_image = cv.normalize(image, dst, 0, 255, cv.NORM_MINMAX)
    return norm_image.flatten()
 
#lookup = {'axes': 1, 'boots': 2, 'carabiners': 3, 'crampons': 4, 'gloves': 5,}
#r_lookup = {1: 'axes', 2: 'boots', 3: 'carabiners', 4: 'crampons', 5: 'gloves'}
lookup = {'axes': 1, 'boots': 2, 'carabiners': 3, 'crampons': 4, 'gloves': 5, 'hardshell_jackets': 6, 'harnesses': 7, 'helmets': 8, 'insulated_jackets': 9, 'pulleys': 10, 'rope': 11, 'tents': 12}
r_lookup = {1: 'axes', 2: 'boots', 3: 'carabiners', 4: 'crampons', 5: 'gloves', 6: 'hardshell_jackets', 7: 'harnesses', 8: 'helmets', 9: 'insulated_jackets', 10: 'pulleys' , 11: 'rope', 12: 'tents'}
img_root = '/home/team12/naresh/gear_images'
img_dirs = [os.path.join(img_root, o) for o in os.listdir(
    img_root) if os.path.isdir(os.path.join(img_root, o))]
class_index = 1
dataset = np.empty((1, desired_size * desired_size * 3))
labelset = np.empty(1)
for img_dir in img_dirs:
    img_files = [f for f in listdir(img_dir) if isfile(join(img_dir, f))]
    image_label = np.full([len(img_files)], lookup[basename(img_dir)])
    #print(image_label)
    image_data = np.empty([len(img_files), desired_size * desired_size * 3])
    index = 0
    for img_file in img_files:
        image_path = img_dir + '/' + img_file
        image_data[index] = normalize(image_path)
        # print(image_data.shape)
        index = index + 1
    class_index = class_index + 1
    dataset = np.concatenate((dataset, image_data), axis=0)
    labelset = np.concatenate((labelset, image_label), axis=0)
 
#print(dataset.astype('uint8').dtype)
#print(labelset.astype('uint8').dtype)
 
(trainData, testData, trainLabels, testLabels) = train_test_split(
    dataset.astype('uint8'), labelset.astype('uint8'), test_size=0.25, random_state=42)
 
(trainData, valData, trainLabels, valLabels) = train_test_split(
    trainData, trainLabels, test_size=0.1, random_state=84)
 
# Checking sizes of each data split
print("training data points: {}".format(len(trainLabels)))
print("validation data points: {}".format(len(valLabels)))
print("testing data points: {}".format(len(testLabels)))
 
kVals = range(1, 30, 2)
accuracies = []
 
# loop over kVals
for k in range(1, 30, 2):
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainData, trainLabels)
 
    # evaluate the model and print the accuracies list
    score = model.score(valData, valLabels)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)
 
# largest accuracy
# np.argmax returns the indices of the maximum values along an axis
i = np.argmax(accuracies)
print("k=%d achieved highest accuracy of %.2f%% on validation data" % (kVals[i],
                                                                       accuracies[i] * 100))
 
# Now that I know the best value of k, re-train the classifier
model = KNeighborsClassifier(n_neighbors=kVals[i])
model.fit(trainData, trainLabels)
 
# Predict labels for the test set
predictions = model.predict(testData)
 
# Evaluate performance of model for each of the digits
print("EVALUATION ON TESTING DATA")
print(classification_report(testLabels, predictions))

for i in np.random.randint(0, high=len(testLabels), size=(5,)):
    image = testData[i]
    prediction = model.predict(image.reshape(1, -1))[0]
    # show the prediction
    print("Image Name {}.png".format(i))
    print("I think that image is: {}".format(r_lookup[prediction]))
    img = Image.fromarray(testData[i].reshape(128,128,3), 'RGB')
    #img.save("{}.png".format(i))
 
test_image = normalize('/home/team12/naresh/test.jpeg')
prediction = model.predict(test_image.reshape(1, -1))[0]
print("I think that image is: {}".format(r_lookup[prediction]))


def showimage(image_path):
    img = Image.open(image_path).convert('RGB')
    plt.imshow(img)    
    plt.show()